In [ ]:
# Ini hanya untuk yang dari spreadsheet region aja ya :)
# Data dashboard sesuai dngn yg diupload, bkn sesuai scope di bawah. Kalau mau sama, maka yg didownload tglnya jg harus sama.
TanggalAwal = '2025-06-30'
TanggalAkhir = '2025-07-16'

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
from google.colab import files
import gspread
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe
import numpy as np
import pandas as pd
from dateutil.parser import parse

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
uploaded = files.upload()

In [ ]:
# Read Excel
filename = next(iter(uploaded))
CT_dashboard_df = pd.read_excel(filename)
CT_dashboard_df = CT_dashboard_df[CT_dashboard_df.iloc[:, 1].notna()]
CT_dashboard_df['Kode Depo'] = CT_dashboard_df['Plants'].str.extract(r'^(\d+)').astype(int)
CT_dashboard_df['Nama Depo'] = CT_dashboard_df['Plants'].str.split('-', n=1).str[1]

# Convert check to score
CT_dashboard_df["Total Waktu Di Jalan > 240 Mins"] = CT_dashboard_df["Total Waktu Di Jalan > 240 Mins"].apply(lambda x: 2 if x == "✅" else 0)
CT_dashboard_df["Mulai > 08:45"] = CT_dashboard_df["Mulai > 08:45"].apply(lambda x: 2 if x == "✅" else 0)
CT_dashboard_df["Selesai < 15:00 (untuk Sabtu, <13:30)"] = CT_dashboard_df["Selesai < 15:00 (untuk Sabtu, <13:30)"].apply(lambda x: 2 if x == "✅" else 0)
CT_dashboard_df["Durasi Kerja < 360 Mins (untuk Sabtu, <300 Min)"] = CT_dashboard_df["Durasi Kerja < 360 Mins (untuk Sabtu, <300 Min)"].apply(lambda x: 3 if x == "✅" else 0)
CT_dashboard_df["Titik Toko > 5 Di Satu Tempat"] = CT_dashboard_df["Titik Toko > 5 Di Satu Tempat"].apply(lambda x: 5 if x == "✅" else 0)
CT_dashboard_df["Foto Selfie Buram / Blank"] = CT_dashboard_df["Foto Selfie Buram / Blank"].apply(lambda x: 3 if x == "✅" else 0)

# Read Spreadsheet - Join Depo-Region
spreadsheet = gc.open("Daily Poin CT")
sheetDepoRegion = spreadsheet.worksheet("DEPO REGION ")
DepoRegion = get_as_dataframe(sheetDepoRegion, evaluate_formulas=True)
df_left = CT_dashboard_df
df_merged = pd.merge(df_left, DepoRegion, left_on='Kode Depo', right_on = 'DEPO', how = 'left')

# Read Spreadsheet - Join Salesman-ASS
sheetASS = spreadsheet.worksheet("ASS")
ASS = get_as_dataframe(sheetASS, evaluate_formulas=True)
df_left = df_merged
df_merged2 = pd.merge(df_left, ASS, left_on='Salesman Id', right_on = 'Kode Salesman', how = 'left')
df_all = df_merged2.copy()

# pivot all
pivotall = df_all.pivot(index=['REGION', 'Kode Depo', 'Nama Depo', 'ASS', 'Salesman Id', 'Salesman Name', 'NIK ASS', 'Nama ASS'], columns='Date', values='Total Points').reset_index()

# upload pivot all to sheet
sheetnew = spreadsheet.worksheet("POIN CT")
sheetnew.clear()
set_with_dataframe(sheetnew, pivotall)

# pivot-upload masing-masing penilaian
sheet_name = ["Total Waktu Di Jalan > 240 Mins", "Mulai > 08:45", "Selesai < 15:00 (untuk Sabtu, <13:30)", "Durasi Kerja < 360 Mins (untuk Sabtu, <300 Min)", "Titik Toko > 5 Di Satu Tempat", "Foto Selfie Buram / Blank"]
for name in sheet_name:
  pivot = df_all.pivot(index=['REGION', 'Kode Depo', 'Nama Depo', 'ASS', 'Salesman Id', 'Salesman Name', 'NIK ASS', 'Nama ASS'], columns='Date', values=name).reset_index()
  sheetnew = spreadsheet.worksheet(name)
  sheetnew.clear()
  set_with_dataframe(sheetnew, pivot)


spreadsheet_name = ["R01_SUMUT_CT_2.0", "R02_SUMBAR_CT_2.0", "R03_SUMTENG_CT_2.0", "R04_SUMSEL_CT_2.0", "R05_BANTEN_CT_2.0", "R06_JABODEBEK_CT_2.0", "R07_BANDUNGSUCI_CT_2.0", "R08_PANTURA_CT_2.0", "R09_PRIANGAN TIMUR_CT_2.0", "R10_JATENG UTARA_CT_2.0", "R11_JATENG SELATAN_CT_2.0", "R12_JATIM UTARA_CT_2.0", "R13_JATIM TENGAH_CT_2.0", "R14_JATIM SELATAN_CT_2.0", "R15_BALI NUSRA_CT_2.0", "R16_KALBARTIM_CT_2.0", "R17_KALTENGSEL_CT_2.0", "R18_SULAWESI 1_CT_2.0", "R19_SULAWESI 2_CT_2.0", "R20_SULAWESI 3_CT_2.0"]

# Mapping Bahasa month names to English
month_map = {
  'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'Mei': 'May',
  'Jun': 'Jun', 'Jul': 'Jul', 'Agu': 'Aug', 'Sep': 'Sep', 'Okt': 'Oct',
  'Nov': 'Nov', 'Des': 'Dec'
}

def translate_month(date_str):
    for indo, eng in month_map.items():
        if indo in date_str:
            return date_str.replace(indo, eng)
    return date_str

def safe_parse(val):
    try:
        translated = translate_month(val)
        return parse(translated, dayfirst=True)
    except (ValueError, TypeError):
        return val

for name in spreadsheet_name:
    sheet = gc.open(name).worksheet("Pengerjaan Harian - ASS")
    data = sheet.get_all_values()
    df = pd.DataFrame(data)
    subset = df[df[3] != ""]
    subset = subset[1:]
    subset[59] = name
    print(name)
    if name == "R01_SUMUT_CT_2.0":
        template = subset
    else:
        template = pd.concat([template, subset], ignore_index=True)


def clean_cell(val):
    if isinstance(val, str) and val.startswith("'") and val.endswith("'"):
        stripped = val[1:-1].strip()

        # Cek integer
        if stripped.isdigit():
            return int(stripped)

        # Cek boolean
        lowered = stripped.lower()
        if lowered == 'true':
            return True
        elif lowered == 'false':
            return False

        # Cek tanggal dengan berbagai format
        date_formats = ['%Y-%m-%d', '%d %B %Y', '%d %b %Y']
        for fmt in date_formats:
            try:
                return datetime.strptime(stripped, fmt)
            except ValueError:
                continue

    return val

all_data = template.applymap(clean_cell)
all_data[2] = all_data[2].apply(safe_parse)
all_data[2] = pd.to_datetime(all_data[2], errors='coerce')
filter_data = all_data[(all_data[2] >= pd.to_datetime(TanggalAwal)) & (all_data[2] <= pd.to_datetime(TanggalAkhir))]
filter_data["Foto NPL minimal 5 foto/hari"] = filter_data[16].apply(lambda x: 1 if x == True else 0)
filter_data["Foto min. 5 pajangan Nabati/hari"] = filter_data[17].apply(lambda x: 1 if x == True else 0)
filter_data["SMD Minimal 3 Call dan WAJIB Foto Rak Pajangan"] = filter_data[18].apply(lambda x: 1 if x == True else 0)

filter_data[2] = pd.to_datetime(filter_data[2])
# filter_data[7] = filter_data[7].astype(int)
filter_data[7] = pd.to_numeric(filter_data[7], errors='coerce').fillna(0).astype(int)
df_merged2['Date'] = pd.to_datetime(df_merged2['Date'])

filter_data_unique = filter_data.drop_duplicates(subset=[7, 2])
df_merged3 = pd.merge(df_merged2, filter_data_unique[[7, 2, 16, 17, 18]], left_on=['Salesman Id', 'Date'], right_on = [7, 2], how = 'left')
df_merged3.rename(columns={16: 'Foto NPL minimal 5 foto/hari', 17: 'Foto min. 5 pajangan Nabati/hari', 18:'SMD Minimal 3 Call dan WAJIB Foto Rak Pajangan'}, inplace=True)

# pivot-upload masing-masing penilaian
sheet_name = ['Foto NPL minimal 5 foto/hari', 'Foto min. 5 pajangan Nabati/hari', 'SMD Minimal 3 Call dan WAJIB Foto Rak Pajangan']
for name in sheet_name:
  pivot = df_merged3.pivot_table(index=['REGION', 'Kode Depo', 'Nama Depo', 'ASS', 'Salesman Id', 'Salesman Name', 'NIK ASS', 'Nama ASS'], columns=2, values=name, aggfunc = 'first' ).reset_index()
  sheetnew = spreadsheet.worksheet(name)
  sheetnew.clear()
  set_with_dataframe(sheetnew, pivot)